# Install the required packages

In [66]:
import sys
import os
import glob
import matplotlib.pyplot as plt

import numpy as np
#to work with text files
import pandas as pd 

#pre-processing imports
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
import nltk
nltk.download('stopwords') # stopword 
nltk.download('punkt') # word_tokenize

#imports related to modeling
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# word2vec installation
- There is no need to change the video texts to lower case as there are some words are capital letters in **word2vec**

In [67]:
!brew install wget
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
data_path= "DATAPATH"
%time 
w2v_model = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True)
print('done loading Word2Vec')

/bin/bash: brew: command not found
--2021-11-22 04:48:48--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.94.21
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.94.21|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 12.2 µs
done loading Word2Vec


In [68]:
# Inspect the model
word2vec_vocab = w2v_model.vocab.keys()
word2vec_vocab_lower = [item.lower() for item in word2vec_vocab]
word2vec_vocab_show = [item for item in word2vec_vocab]
print(len(word2vec_vocab))
print(word2vec_vocab_show[500:1000])

3000000
['doing', 'face', 'low', 'higher', 'site', 'once', 'yet', 'hours', 'America', 'control', 'received', 'rate', 'career', 'Bush', 'teams', 'known', 'offer', 'race', 'ever', 'experience', 'playing', 'name', 'possible', 'countries', 'Mr.', 'average', 'together', 'using', '9', 'cut', 'While', 'total', 'round', 'young', 'nearly', 'shares', 'member', 'campaign', 'media', 'needs', 'why', 'house', 'issues', 'costs', 'fire', '##-#', 'victory', 'player', 'began', 'sure', 'story', 'per_cent', 'North', 'His', 'staff', 'order', 'war', 'large', 'interest', 'stock', 'food', 'research', 'key', 'India', 'South', 'morning', 'conference', 'senior', 'global', 'Center', 'death', 'person', 'thought', 'gave', 'feel', 'energy', 'history', 'recently', 'largest', 'No.', 'general', 'official', 'released', 'wanted', 'meet', 'short', 'outside', 'running', 'live', 'ball', 'online', 'real', 'position', 'fact', 'fell', 'nine', 'December', 'front', 'action', 'defense', 'problem', 'problems', 'Mr', 'nation', 'nee

In [69]:
# Ref: https://www.geeksforgeeks.org/python-os-chdir-method/
# Ref: https://stackoverflow.com/questions/9234560/find-all-csv-files-in-a-directory-using-python/12280052 

def get_list_filenames(cwd, path, extension):
    try:
        os.chdir(path)
        print("Successfully change the directory!")
        filenames = glob.glob('*.{}'.format(extension))
        print("Number of files: ", len(filenames))

    # Caching the exception
    except: 
        print("Something wrong with specified directory. Exception- ", sys.exc_info()) 

    # handling with finally: restore the path which is the current directory before changing directory           
    finally: 
        print("Restoring the path") 
        os.chdir(cwd) # Change it back to cwd (original directory)
        print("Current directory is-", os.getcwd()) 
        
    return filenames


# Ref: https://realpython.com/python-keras-text-classification/

# Get a list of dataframe after reading all csv files given a path
def get_all_image_files_read(path, filenames):
    list_img = []
    for file in filenames:
        img = imageio.imread(path+file)
        list_img.append(img)
    return list_img

In [ ]:
%cd /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/fake
!pwd

/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/fake
/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/fake


In [ ]:
# initial directory 
cwd = os.getcwd()
path_org = './' # the directory where all the output result are located.
extension = 'txt'

fake_textfiles = get_list_filenames(cwd, path_org, extension)
print(fake_textfiles[0])
print(len(fake_textfiles))

Successfully change the directory!
Number of files:  4500
Restoring the path
Current directory is- /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/fake
FAKE_BCD3501.txt
4500


- 360 from B fake videos
- 1884 from C fake videos
- 2256 from D fake videos

Totally 4500 text files

In [ ]:
print(fake_textfiles[2300])

FAKE_BCD1801.txt


# Tokenize, Remove Stopwords, and Lowercase Conversion
- Keeping punctuations could be relevant for this task

In [71]:
aaa = []
aaa.append(open(fake_textfiles[786]).readline())
print(aaa)

mystopwords = set(stopwords.words("english"))
# function to tokenize tweets, remove stopwords and numbers.
#k eeping punctuations and emoticon symbols could be relevant for this task!
def preprocess_corpus(texts):
  def remove_stops_digits(tokens):
  #Nested function that removes stopwords and digits from a list of tokens
    return [token for token in tokens if token not in mystopwords and not token.isdigit()]
    #This return statement below uses the above function to process twitter tokenizer output further.
  return [remove_stops_digits(word_tokenize(content.lower())) for content in texts]


bbb = preprocess_corpus(aaa)
flatten_list = sum(bbb, [])
print(flatten_list)


['the walls that every time they play every note has the same amount of weight and import']
['walls', 'every', 'time', 'play', 'every', 'note', 'amount', 'weight', 'import']


# Create **word2vec** Vetorzation Function

In [72]:
# Creating a feature vector by averaging all embeddings for all sentences
def embedding_feats(list_of_lists):
  DIMENSION = 300
  zero_vector = np.zeros(DIMENSION)
  feats = []
  for tokens in list_of_lists:
    feat_for_this = np.zeros(DIMENSION)
    count_for_this = 0
    for token in tokens:
      if token in w2v_model:
        feat_for_this += w2v_model[token]
        count_for_this +=1
    feats.append(feat_for_this/count_for_this)
  return feats


train_vector = embedding_feats(flatten_list)
train_vector = np.array(train_vector)
print(train_vector.shape)
print(train_vector)

flaa = np.ndarray.flatten(train_vector)
print(flaa.shape)
print(flaa)

(9, 300)
[[-1.44714355e-01  1.75292969e-01 -9.45434570e-02 ... -8.18786621e-02
  -1.05834961e-01  1.73706055e-01]
 [-7.82226562e-02  1.15454102e-01  7.42675781e-02 ...  2.44140625e-04
  -1.00439453e-01  2.18457031e-01]
 [-1.96533203e-01  1.25244141e-01 -6.95190430e-02 ...  1.14440918e-02
  -9.20410156e-02  4.53643799e-02]
 ...
 [-1.84631348e-01  9.37255859e-02 -4.13330078e-02 ... -4.37011719e-02
  -8.82324219e-02  5.24169922e-02]
 [-2.56998698e-01  1.12274170e-01 -7.13297526e-02 ... -5.56437174e-02
  -1.49088542e-01  1.38641357e-01]
 [-2.06461589e-01  1.24023438e-01 -6.02416992e-02 ... -1.72119141e-02
  -4.98860677e-02  8.89994303e-02]]
(2700,)
[-0.14471436  0.17529297 -0.09454346 ... -0.01721191 -0.04988607
  0.08899943]


# create `FAKE_RTVC_B` text features
- total frames: 360 videos * 4 frames = 1440

In [76]:
fake_B = []
for i in range(0, 360):
  openlist = []
  openlist.append(open(fake_textfiles[i]).readline())
  # apply to function
  tmp = preprocess_corpus(openlist)
  # faltten 2d list to 1d
  tmp = sum(tmp, [])
  text_vectors = embedding_feats(tmp)
  text_vectors = np.array(text_vectors, dtype=object)
  final = np.ndarray.flatten(text_vectors)
  print(final.shape)
  #del tmp
  fake_B.append(final)

(300,)
(3000,)
(300,)
(300,)
(1500,)
(300,)
(3000,)
(2400,)
(300,)
(2100,)
(1200,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(4800,)
(4800,)
(4800,)
(1500,)
(2700,)
(2700,)
(1500,)
(2700,)
(2700,)
(2700,)
(4800,)
(1500,)
(1500,)
(1500,)
(4800,)
(2700,)
(1500,)
(1500,)
(2700,)
(1500,)
(4800,)
(2700,)
(4800,)
(4800,)
(2700,)
(3300,)
(1500,)
(1500,)
(2700,)
(4800,)
(4800,)
(900,)
(2100,)
(3900,)
(6900,)
(3900,)
(6900,)
(6900,)
(6900,)
(6900,)
(3900,)
(6900,)
(3900,)
(3900,)
(6900,)
(6900,)
(3900,)
(6900,)
(3900,)
(3900,)
(6900,)
(3900,)
(3900,)
(1500,)
(300,)
(1800,)
(3300,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(2700,)
(2700,)
(1200,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(6000,)
(1200,)
(900,)
(2400,)
(2400,)
(2400,)
(2400,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]



(2700,)
(4500,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(1800,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(3900,)
(2700,)
(3900,)
(3900,)
(3900,)
(3900,)
(3900,)
(3900,)
(3900,)
(3900,)
(3900,)
(5100,)
(5100,)
(5100,)
(5100,)
(5100,)
(5100,)
(5100,)
(5100,)
(5100,)
(5100,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(300,)
(900,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(2100,)
(1200,)
(6300,)
(6300,)
(6300,)
(6300,)
(6300,)
(6300,)
(6300,)
(6300,)
(6300,)
(6300,)
(5700,)
(5700,)
(5700,)
(5700,)
(5700,)
(5700,)
(5700,)
(5700,)
(5700,)
(300,)
(5700,)
(6300,)
(6300,)
(6300,)
(300,)
(300,)
(6300,)
(300,)
(300,)
(300,)
(6300,)
(300,)
(300,)
(6300,)
(300,)
(6300,)
(300,)
(6300,)
(6300,)
(6300,)
(2700,)
(300,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(3300,)
(3300,)
(1200,)
(3000,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,

In [ ]:
print(len(fake_B))
#print(fake_B[100])

# Because we capture 4 frame in fake B set, so we need to duplicate the text 4 times 
B = np.concatenate((fake_B, fake_B, fake_B, fake_B), axis = 0)
print(len(B))

360
1440


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
fake_B = np.array(fake_B)
print(fake_B[359].shape)

(1500,)


# Create `FAKE_FSGAN_C` to text features
- total frames: 1884 videos * 2 frames = 3768

In [ ]:
fake_C = []
for i in range(360, 2244):
  tmp = []
  tmp.append(open(fake_textfiles[i]).readline())
  # apply to function
  tmp = preprocess_corpus(tmp)
  # faltten 2d list to 1d
  tmp = sum(tmp, [])
  text_vectors = embedding_feats(tmp)
  text_vectors = np.array(text_vectors, dtype=object)
  final = np.ndarray.flatten(text_vectors)
  print(final.shape)
  del tmp
  fake_C.append(final)

(1500,)
(1500,)
(900,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(600,)
(2100,)
(1500,)
(1500,)
(2400,)
(3600,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(1500,)
(3000,)
(3000,)
(600,)
(5100,)
(3900,)
(1500,)
(4200,)
(4200,)
(1500,)
(1500,)
(3900,)
(3900,)
(1500,)
(3900,)
(1500,)
(3900,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(3900,)
(3900,)
(3900,)
(2400,)
(2400,)
(2400,)
(2100,)
(2400,)
(2400,)
(2100,)
(2400,)
(2400,)
(2400,)
(3300,)
(2400,)
(3300,)
(3300,)
(6900,)
(2400,)
(3900,)
(2400,)
(600,)
(600,)
(3300,)
(600,)
(8100,)
(8100,)
(2100,)
(8100,)
(1500,)
(3900,)
(1500,)
(1500,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


(8100,)
(8100,)
(1500,)
(600,)
(600,)
(2100,)
(5700,)
(8100,)
(600,)
(600,)
(5700,)
(5700,)
(1500,)
(3300,)
(3300,)
(3300,)
(1200,)
(8100,)
(3300,)
(600,)
(600,)
(8100,)
(600,)
(600,)
(600,)
(2400,)
(3900,)
(3900,)
(3300,)
(2400,)
(600,)
(600,)
(3300,)
(2400,)
(3900,)
(3900,)
(5700,)
(5700,)
(3300,)
(3300,)
(3900,)
(3300,)
(8100,)
(3900,)
(2400,)
(3900,)
(2400,)
(2400,)
(2400,)
(8100,)
(2100,)
(3900,)
(2400,)
(2400,)
(2100,)
(3300,)
(2100,)
(3300,)
(3300,)
(3300,)
(3300,)
(3300,)
(600,)
(1500,)
(2100,)
(1500,)
(4500,)
(3600,)
(3300,)
(2400,)
(300,)
(2100,)
(2700,)
(3600,)
(2400,)
(2400,)
(900,)
(6300,)
(2100,)
(1800,)
(1800,)
(2100,)
(4200,)
(2100,)
(1800,)
(3900,)
(5400,)
(1200,)
(3000,)
(3000,)
(1500,)
(900,)
(2700,)
(1200,)
(3300,)
(2400,)
(3300,)
(2100,)
(300,)
(5100,)
(2700,)
(2400,)
(2400,)
(3900,)
(1800,)
(2100,)
(2400,)
(1800,)
(1500,)
(1200,)
(300,)
(900,)
(3000,)
(2400,)
(2100,)
(3900,)
(2400,)
(1200,)
(2700,)
(3000,)
(2700,)
(3900,)
(2400,)
(3600,)
(6000,)
(300,)
(2100,)
(30

In [ ]:
print(len(fake_C))
#print(fake_B[100])

# Because we capture 2 frame in fake C set , so we need to duplicate the text 2 times 
C = np.concatenate((fake_C, fake_C), axis = 0)
print(len(C))

1884
3768


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


# Create `FAKE_FACESWAP_D_W2L` to text features
- total frames: 2256 videos * 2 frames = 4512

In [ ]:
fake_D = []
for i in range(2244, len(fake_textfiles)):
  tmp = []
  tmp.append(open(fake_textfiles[i]).readline())
  # apply to function
  tmp = preprocess_corpus(tmp)
  # faltten 2d list to 1d
  tmp = sum(tmp, [])
  text_vectors = embedding_feats(tmp)
  text_vectors = np.array(text_vectors, dtype=object)
  final = np.ndarray.flatten(text_vectors)
  print(final.shape)
  del tmp
  fake_D.append(final)

(2400,)
(2400,)
(3000,)
(1800,)
(1500,)
(300,)
(900,)
(3300,)
(3300,)
(3000,)
(1800,)
(2400,)
(2400,)
(2400,)
(2100,)
(300,)
(1500,)
(2100,)
(2400,)
(2100,)
(2400,)
(3300,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


(1500,)
(3300,)
(900,)
(1800,)
(300,)
(2400,)
(1500,)
(2700,)
(1800,)
(2400,)
(1500,)
(3300,)
(2100,)
(1200,)
(2100,)
(1500,)
(1500,)
(1500,)
(1200,)
(2400,)
(2400,)
(4800,)
(1500,)
(1500,)
(5700,)
(1200,)
(1500,)
(4800,)
(3600,)
(3000,)
(6300,)
(2100,)
(2700,)
(2100,)
(3000,)
(300,)
(1200,)
(2100,)
(1500,)
(3900,)
(3300,)
(4200,)
(2700,)
(2700,)
(4500,)
(2700,)
(1500,)
(2400,)
(1800,)
(2100,)
(2400,)
(1800,)
(1800,)
(3300,)
(3300,)
(2100,)
(2700,)
(2700,)
(2400,)
(1500,)
(3000,)
(4200,)
(2700,)
(3600,)
(2100,)
(2100,)
(300,)
(900,)
(2400,)
(2100,)
(1800,)
(3300,)
(3600,)
(1500,)
(4200,)
(2100,)
(1200,)
(1800,)
(2400,)
(2100,)
(3300,)
(3300,)
(1800,)
(2400,)
(1800,)
(1200,)
(2100,)
(1500,)
(2400,)
(1200,)
(2400,)
(300,)
(1200,)
(300,)
(3900,)
(3300,)
(2400,)
(1200,)
(2400,)
(300,)
(1500,)
(2700,)
(2700,)
(2100,)
(300,)
(2700,)
(1200,)
(2400,)
(300,)
(3000,)
(300,)
(1200,)
(4200,)
(3600,)
(2700,)
(300,)
(1200,)
(1800,)
(3300,)
(1800,)
(3000,)
(2700,)
(3900,)
(2700,)
(2100,)
(3900,)
(180

In [ ]:
print(len(fake_D))
#print(fake_B[100])

# Because we capture 2 frame in fake D set , so we need to duplicate the text 2 times 
D = np.concatenate((fake_D, fake_D), axis = 0)
print(len(D))

2256
4512


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


# Save BCD  as fake `npy file`
- The number of fake text observations is 9720 

In [ ]:
B, C, D = np.array(B), np.array(C), np.array(D)
fake_texts = np.concatenate((B, C, D), axis = 0)

print(len(fake_texts))
print(fake_texts[1000].shape)
print(fake_texts[2000]].shape)
print(fake_texts[1000].shape)
print(fake_texts[1000].shape)

9720
(5700,)


In [ ]:
print(fake_texts[300])

[-0.0926666259765625 -0.015594482421875 -0.026214599609375 ...
 -0.029500325520833332 -0.13818359375 0.16050211588541666]


In [ ]:
%cd /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/np_data
!pwd
np.save("fake_texts.npy", fake_texts)

/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/np_data
/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/np_data


# Create `REAL_A` text features

- total **REAL** images: 390 videos * 25 frames = 9750

In [ ]:
%cd /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/real
!pwd

/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/real
/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/real


In [ ]:
# initial directory 
cwd = os.getcwd()
path_org = './' # the directory where all the output result are located.
extension = 'txt'

real_textfiles = get_list_filenames(cwd, path_org, extension)
print(real_textfiles[0])
print(len(real_textfiles))

Successfully change the directory!
Number of files:  390
Restoring the path
Current directory is- /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/real
REAL_A0.txt
390


In [ ]:
real_A = []
for i in range(len(real_textfiles)):
  tmp = []
  tmp.append(open(real_textfiles[i]).readline())
  # apply to function
  tmp = preprocess_corpus(tmp)
  # faltten 2d list to 1d
  tmp = sum(tmp, [])
  text_vectors = embedding_feats(tmp)
  text_vectors = np.array(text_vectors, dtype=object)
  text_vectors = np.ndarray.flatten(text_vectors)
  print(text_vectors.shape)
  del tmp
  real_A.append(text_vectors)

(1500,)
(1500,)
(1500,)
(4500,)
(4500,)
(3900,)
(1500,)
(4500,)
(1500,)
(3900,)
(4500,)
(4500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(300,)
(300,)
(1500,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(3300,)
(2100,)
(2100,)
(2100,)
(1500,)
(2100,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(900,)
(900,)
(1500,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(3000,)
(900,)
(3000,)
(3000,)
(900,)
(900,)
(3000,)
(2100,)
(3000,)
(3000,)
(3000,)
(2100,)
(2100,)
(2100,)
(3000,)
(3000,)
(2100,)
(2100,)
(2100,)
(1200,)
(1200,)
(1500,)
(1500,)
(600,)
(2100,)
(2100,)
(1500,)
(1200,)
(1200,)
(1200,)
(1200,)
(1200,)
(1200,)
(1200,)
(1200,)
(1200,)
(1200,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(1800,)
(1800,)
(5100,)
(4200,)
(2700,)
(2700,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(900,)
(4200,)
(4200,)
(1200,)
(4500,)
(1500,)
(900,)
(4200,)
(2400,)
(1800,)
(900,)
(1500,)
(900,)
(1800,)
(1500,)
(900,)
(1500,)
(3300,)
(3300,)
(3300,)
(3300,)
(3300,)
(3300,)
(3300,)
(3300,)
(3300,)
(900,)
(900,)
(300,)
(4200,)
(1200,)
(1500,)
(1200,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(3900,)
(1500,)
(1500,)
(1500,)
(2700,)
(2100,)
(4800,)
(1800,)
(2100,)
(4800,)
(4800,)
(4800,)
(4800,)
(4800,)
(4800,)
(4800,)
(1800,)
(1800,)
(4800,)
(1800,)
(4800,)
(1800,)
(1800,)
(2700,)
(2400,)
(2400,)
(1800,)
(2400,)
(1800,)
(1800,)
(5400,)
(1800,)
(2400,)
(1800,)
(1800,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2400,)
(3000,)
(300,)
(2100,)
(3000,)
(2100,)
(300,)
(3

In [ ]:
A = np.array(real_A, dtype=object)
real_texts = np.concatenate((A, A, A, A, A,
                             A, A, A, A, A,
                             A, A, A, A, A,
                             A, A, A, A, A,
                             A, A, A, A, A), axis = 0)
print(len(real_texts))
print(real_texts.shape)

9750
(9750,)


# Save A as real `npy file`
- The number of fake text observations is 9750

In [ ]:
%cd /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/np_data
!pwd
np.save("real_texts.npy", real_texts)

/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/np_data
/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/np_data


# Concatenate fake and real arrays then save as `npy` file

For $X_2$ text features, we have
- real video: 9750 observations from `Real_A`.
- fake videos: 9720 observations from `FAKE_RTVC_B`, `FAKE_FSGAN_C` and `FAKE_FACESWAP_D_W2L`.
- The number of observations is 19470

In [ ]:
X2_texts = np.concatenate((fake_texts, real_texts), axis = 0)
print(len(X2_texts))
print(X2_texts.shape)
np.save("X2_texts.npy", X2_texts)

19470
(19470,)


# References
- https://code.google.com/archive/p/word2vec/